### 解决了1): 判断文件统计口径是时间点or时间段；2): 从第n列读取Excel表。

In [1]:
import os, re
import numpy as np
import pandas as pd

In [2]:
# 工具函数
def dataDate_mode(file_name):
    '''
    判断文件统计口径是时间点or时间段
    '''
    file_name = file_name[:file_name.find('.')]
    file_name_ls = file_name.split('_')
    
    TF_list = []
    # 正则匹配yyyyMMdd格式
    pattern = '((\d{3}[1-9]|\d{2}[1-9]\d|\d[1-9]\d{2}|[1-9]\d{3})(((0[13578]|1[02])(0[1-9]|[12]\d|3[01]))|((0[469]|11)(0[1-9]|[12]\d|30))|(02(0[1-9]|[1]\d|2[0-8]))))|(((\d{2})(0[48]|[2468][048]|[13579][26])|((0[48]|[2468][048]|[3579][26])00))0229)'
    for param in file_name_ls:
        if bool(re.match(pattern, param, flags=0)):
            TF_list.append(param)
        
    return TF_list

def excel_to_sql_timePoint(file_name,TF_list,table_name,skiprows):
    '''
    按时间点的模版插入SQL语句
    '''
    Data_Date = TF_list[0]
    
    df = pd.read_excel(file_name,skiprows=skiprows)
    Table_Header = pd.DataFrame({'ID':np.nan * len(df), 'DATA_DATE':[Data_Date]*len(df)})
    df = pd.concat([Table_Header, df], axis=1)  
    
    cols_len = len(df.iloc[0])
    exec_code_ls = []
    
    for j in range(len(df)):
        col_values = 'INSERT INTO ' + table_name + ' values('
        # 将"乐观锁",“创建人”,“创建时间”,“更新人”,“更新时间列”设置为空值
        for i in range(cols_len):
            if i == 2:
                col_values += 'NULL,'*5

            col_values += isNull(df.iloc[j][i])

        # 去除最后一个逗号
        col_values = col_values[:-1]

        # 加上后缀
        col_values += ');'
        exec_code_ls.append(col_values)
    
    return exec_code_ls
    

def excel_to_sql_timeQuantum(file_name,TF_list,table_name,skiprows):
    '''
    按时间段的模版插入SQL语句
    '''
    Data_Start_Date = min(TF_list)
    Data_End_Date = max(TF_list)
    
    df = pd.read_excel(file_name,skiprows=skiprows)
    Table_Header = pd.DataFrame({'ID':np.nan * len(df), 'DATA_START_DATE':[Data_Start_Date]*len(df)})
    df = pd.concat([Table_Header, df], axis=1)
    end_date = pd.DataFrame({'DATA_END_DATE':[Data_End_Date]*len(df)})
    df = pd.concat([df,end_date], axis=1)
    
    cols_len = len(df.iloc[0])
    exec_code_ls = []
    
    for j in range(len(df)):
        col_values = 'INSERT INTO ' + table_name + ' values('
        # 将"乐观锁",“创建人”,“创建时间”,“更新人”,“更新时间列”设置为空值
        for i in range(cols_len):
            if i == 2:
                col_values += 'NULL,'*5

            col_values += isNull(df.iloc[j][i])

        # 去除最后一个逗号
        col_values = col_values[:-1]

        # 加上后缀
        col_values += ');'
        exec_code_ls.append(col_values)
    
    return exec_code_ls

def isNull(values):
    error_type = ['nan', 'NaN', 'NaT']
    if str(values) in error_type:
        output = 'NULL,'
    else:
        output = '\'' + str(values) + '\','
    return output

In [3]:
FN1 = '交强险追偿清单_20210131.xls'
FN2 = '相同两车多次双方事故_全司_20201201_20201231.xlsx'
FN3 = '车理赔老旧车型赔案_安徽分公司_20190101_20201231.xlsx'

table_name1 = '交强险追偿清单'
table_name2 = '相同两车多次双方事故'
table_name3 = '车理赔老旧车型赔案'

In [4]:
def excel_to_sql_mode(file_name,table_name,skiprows):
    TF_list = dataDate_mode(file_name)
    # 统计口径为时间点
    if len(TF_list) == 1:
        exec_code_ls = excel_to_sql_timePoint(file_name,TF_list,table_name,skiprows)
    # 统计口径为时间段
    elif len(TF_list) == 2:
        exec_code_ls = excel_to_sql_timeQuantum(file_name,TF_list,table_name,skiprows)
    else:
        print("文件名异常!")
    
    return exec_code_ls

#### 测试读取不同类型文件结果:

The kener below may not be execute.

In [6]:
exec_code_ls = excel_to_sql_mode(FN1, table_name1,0)
print(exec_code_ls[0])

INSERT INTO 交强险追偿清单 values(NULL,'20210131',NULL,NULL,NULL,NULL,NULL,'1','1010100','北京分公司','北京分公司','顺义支公司','C110113VEH20012552','M110113CTP20005694','M110113CTP20005694_3','杨威','2020-12-08 17:40:49','2020-12-08 18:03:01','2021-01-21 15:21:47','2000.0',NULL,NULL,'赔款退回追偿','否','m','20210131','2021-02-01 03:55:14',NULL,NULL);


In [7]:
exec_code_ls = excel_to_sql_mode(FN2, table_name2,0)
print(exec_code_ls[0])

INSERT INTO 相同两车多次双方事故 values(NULL,'20201201',NULL,NULL,NULL,NULL,NULL,'202012','北京分公司','北京分公司','东城支公司','11022400','交强险','ABEJ020CTP18B163441U','2019-01-17 00:00:00','2020-01-16 00:00:00','122000','587.45','122000','587.45','2018-12-18 00:00:00','2018-12-18 15:42:02','新保','有效','王佳琦','PQ229','554.2','587.45','0',NULL,NULL,'张校英','京NZ3V66','LVHFA1655A5102181','思域DHW7180B(CIVIC 1.8)轿车','2010-12-20 00:00:00','8','京NW51E6','110111198708022210','于银','130435198508081823','张校英',NULL,'130435198508081823','张校英',NULL,'130435198508081823','张校英',NULL,'2019-04-09 23:00:44','2019-04-10 20:40:58','2019-04-13 17:02:38','2019-04-14 10:12:01','2019-04-14 10:25:33','张校英','15801188214','15801188214','C110101VEH19006424','M110101CTP19002420','M110101CTP19002420_1','已立案','已赔付','N','京NZ3V66 非已在 标的车：京NZ3V66右侧受损，撞机动车，碰三者车：京NW51E6左侧受损，已报警或提示报警；非现','海淀区','130435198508081823','张校英',NULL,NULL,NULL,'否',NULL,'800','0','800','2019-04-15 00:00:00','北京圆明园东路汽车维修有限公司','91110154800003757','上海浦东发展银行北京阜成支行',NULL,NULL,NULL,NUL

In [8]:
exec_code_ls = excel_to_sql_mode(FN3, table_name3,3)
print(exec_code_ls[0])

INSERT INTO 车理赔老旧车型赔案 values(NULL,'20190101',NULL,NULL,NULL,NULL,NULL,'201901','安徽分公司','合肥中心支公司','综合业务部','11024400','机动车商业行业示范汽车保险','AHEFHF1Y1418B017886E',NULL,NULL,'2018-04-02 00:00:00','2019-04-01 00:00:00','1076768','4011.75','皖A87V50','LVSHBFMC4BF188531','蒙迪欧CAF7203M轿车','2011-09-21 00:00:00','7','340123199112252077','吴海洲','340123199112252077','吴海洲',NULL,NULL,'C340100VEH18038623','皖A87V50 标的车：皖A87V50右侧受损，撞非机动车/固定物/动物，碰三者物损：护栏有损，已报警或提示报警；现',NULL,NULL,NULL,'否','否',NULL,NULL,NULL,'安徽省滁州市定远县合徐高速',NULL,'吴海洲','F340100A1418036692','F340100A1418036692_4','64435','0','2019-01-09 00:00:00','0','吴海洲','64435','徽商银行股份有限公司阜阳颍东支行','6217751020000257413','2019-01-10 00:00:00','54560','76768','AH3401000191','合肥恒信德龙美华汽车销售服务有限公司','AH3401000191','合肥恒信德龙美华汽车销售服务有限公司',NULL,NULL,NULL,NULL,'HF796','管如晟',NULL,NULL,'BB224','陈子明','HF1969',NULL,'HF902','吴凡','自理自核',NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,'20201231');
